In [3]:
pip install langchain_community langchain_openai faiss-cpu pypdf

Note: you may need to restart the kernel to use updated packages.


In [1]:
import openai
import json

# OpenAI API 클라이언트 설정
client = openai.Client(api_key = "api_key")  # API 키 설정

# JSON 파일 로드
json_file_path = "./json/all_data.json"

with open(json_file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# RAG 기반 분류 기준 (각 분야의 특징)
classification_criteria = {
    "인문": "인간의 존재와 관련된 문제, 그리고 인간의 사상과 문화 등을 다루고 있는 글이다. 인간의 본질이나 정신세계, 그리고 인간의 행위에 대한 이해를 목적으로 하는 글이다. 인간과 세계의 본질과 관련된 글, 인간의 행위 규범과 관련된 글, 인간의 의식 세계와 관련된 글, 사유의 형식이나 법칙과 관련된 글, 그리고 역사나 종교와 관련된 글 등을 포함한다.",
    "예술": "미의 본질이나 미를 추구하는 인간의 다양한 예술 행위에 대해 다루고 있는 글이다. 예술의 본질과 다양한 예술 행위의 특징을 이해하는 한편, 예술 작품을 수용하는 미적 안목을 향상하는 데 도움을 주기 위한 글이다. 예술의 본질에 대해 논의하는 글, 다양한 예술 행위의 특징을 설명하는 글, 주요 예술가나 예술 작품을 비평하는 글, 예술 사조에 대해 설명하는 글 등을 포함한다.",
    "사회": "사회에서 일어나거나 일어날 수 있는 다양한 문제를 소개하고 해결하는 방안을 제시하 는 글이다. 사회 현상이나 문화 현상을 다양한 관점에서 논리적, 체계적으로 설명하는 글이다. 법을 다룬 법학, 사회 제도 및 사회의 다양한 현상을 연구하는 사회학, 기업의 경영을 다룬 경영학, 경제 문제 및 경제 활동을 설명하는 경제학, 생물로서의 인간을 종합적으로 연구하는 인류학, 사회 구성원에 의해 이루어진 생활 양식 및 그와 관련하여 일어나는 여러 현상들을 연구하는 문화학과 관련된 글 등을 포함한다.",
    "기술": "인간의 삶을 편리하게 하는 산업 기술, 생활 기술 등 다양한 분야의 기술을 설명하는 글이다. 특정 과학 이론을 바탕으로 장치나 시스템에 적용되는 원리와 작동 과정, 한계 등을 구체적으로 서술한 글이다. 전기와 전자의 원리를 이용한 공학 기술, 컴퓨터를 이용한 공학 기술, 화학이나 생명 과학과 결합된 공학 기술, 토목이나 건축에 활용되는 토목건축 공학 기술과 관련된 글 둥을 포함한다.",
    "과학": "자연 과학적 시각으로 물질계와 생태계, 우주를 탐구하는 인간의 정신 활동을 담고 있는 글이다. 수에 관하여 연구하는 수학, 물질의 물리적 성질과 운동 형태 등을 연구하는 물리학, 물질의 조성과 구조 · 성질 등을 연구하는 화학, 생물의 구조와 기능을 과학적으로 연구하는 생명 과학, 지구 및 천체를 연구하는 지구 과학과 관련된 글 등을 포함한다.",
}


def classify_subject_and_topics(passage):
    """passage를 분석하여 subject(5가지 주제 중 하나)와 topics(핵심 내용 요약)를 반환하는 함수"""
    prompt = f"""
    아래의 글을 5가지 주제 중 하나로 분류하고, 핵심 키워드를 15자 이내로 요약해주세요.

    --- 주제 분류 기준 ---
    {json.dumps(classification_criteria, indent=4, ensure_ascii=False)}

    --- 글 ---
    {passage}

    위 글의 주제는 무엇인가요? 한 단어로만 답변해주세요. (인문, 예술, 사회, 기술, 과학 중 선택)
    
    또한, 글의 핵심 내용을 15자 이내로 요약한 키워드를 1~3개 뽑아주세요. 쉼표(,)로 구분해주세요.

    출력 형식:
    주제: [주제명]
    키워드: [키워드1], [키워드2], [키워드3]
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "당신은 주어진 글을 적절한 주제(인문, 예술, 사회, 기술, 과학)로 분류하고, 핵심 키워드를 추출하는 AI입니다."},
                  {"role": "user", "content": prompt}],
        temperature=0.2
    )

    output = response.choices[0].message.content.strip()
    
    # 결과 파싱
    lines = output.split("\n")
    subject = lines[0].replace("주제: ", "").strip()
    topics = lines[1].replace("키워드: ", "").strip().split(", ")

    return subject, topics


# JSON 데이터 업데이트
for entry in data:
    passage = entry["passage"]
    
    # subject와 topics 추출
    subject, topics = classify_subject_and_topics(passage)
    
    # topics가 비어있을 경우 최대 2번까지 재시도
    retry_count = 0
    while not topics and retry_count < 2:
        subject, topics = classify_subject_and_topics(passage)
        retry_count += 1
    
    # 최종 값 저장
    entry["subject"] = subject
    entry["topics"] = topics 



# 결과 저장
output_json_path = "./subject_topic_filtered.json"

with open(output_json_path, "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print(f"분류가 완료된 JSON이 저장되었습니다: {output_json_path}")



분류가 완료된 JSON이 저장되었습니다: ./subject_topic_filtered.json


In [4]:
## question_type 

# JSON 파일 로드
json_file_path = "./subject_topic_filtered.json"

with open(json_file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# 질문 유형 기준
question_type_criteria = {
    "사실적 읽기": [
        {"pattern": "내용 전개 방식으로 적절한 것은", "type": "정답형"},
        {"pattern": "내용 전개 방식으로 가장 적절한 것은", "type": "최선답형"},
        {"pattern": "내용 전개 방식으로 적절하지 않은 것은", "type": "부정형"},
        {"pattern": "내용과 일치하는 것은", "type": "정답형"},
        {"pattern": "내용과 일치하지 않는 것은", "type": "부정형"},
        {"pattern": "대한 이해로 적절한 것은", "type": "정답형"},
        {"pattern": "대한 이해로 가장 적절한 것은", "type": "최선답형"},
        {"pattern": "대한 이해로 적절하지 않은 것은", "type": "부정형"},
    ],
    "추론적 읽기": [
        {"pattern": "바탕으로 <보기>를 이해한 내용으로 적절한 것은", "type": "정답형"},
        {"pattern": "바탕으로 <보기>를 이해한 내용으로 가장 적절한 것은", "type": "최선답형"},
        {"pattern": "바탕으로 <보기>를 이해한 내용으로 적절하지 않은 것은", "type": "부정형"},
        {"pattern": "필자의 관점에서 <보기>를 평가한 내용으로 적절한 것은", "type": "정답형"},
        {"pattern": "필자의 관점에서 <보기>를 평가한 내용으로 가장 적절한 것은", "type": "최선답형"},
        {"pattern": "필자의 관점에서 <보기>를 평가한 내용으로 적절하지 않은 것은", "type": "부정형"},
    ],
    "비판적 읽기": [
        {"pattern": "본문의 \(ㄱ\)에 대해 보인 반응으로 적절한 것은", "type": "정답형"},
        {"pattern": "본문의 \(ㄱ\)에 대해 보인 반응으로 가장 적절한 것은", "type": "최선답형"},
        {"pattern": "본문의 \(ㄱ\)에 대해 보인 반응으로 적절하지 않은 것은", "type": "부정형"},
    ]
}

def classify_question_type(question_text):
    """주어진 질문을 기준에 맞춰 question_type을 분류하는 함수"""
    for category, rules in question_type_criteria.items():
        for rule in rules:
            if rule["pattern"] in question_text:
                return f"{category} - {rule['type']}"
    return "기타"

# JSON 데이터 업데이트
for entry in data:
    question_text = entry.get("question_text", "")
    entry["question_type"] = classify_question_type(question_text)

# 결과 저장
output_json_path = "./information_added.json"

with open(output_json_path, "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print(f"질문 유형 분류가 완료된 JSON이 저장되었습니다: {output_json_path}")

TypeError: unhashable type: 'dict'

In [8]:
!pip install rapidfuzz

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 9.6 MB/s eta 0:00:00


In [15]:
## question_type 
from rapidfuzz import process

# JSON 파일 로드
json_file_path = "./subject_topic_filtered.json"

with open(json_file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# 질문 유형 기준
question_type_criteria = {
    "윗글에 대한 이해로 적절한 것은?": [
        "윗글에 대한 이해로 적절한 것은?",
        "윗글에 대한 설명으로 적절한 것은?",
        "윗글의 내용과 일치하는 것은?"
    ],
    "윗글에 대한 이해로 가장 적절한 것은?": [
        "윗글에 대한 이해로 가장 적절한 것은?",
        "윗글에 대한 설명으로 가장 적절한 것은?"
    ],
    "윗글에 대한 이해로 적절하지 않은 것은?": [
        "윗글에 대한 이해로 적절하지 않은 것은?",
        "윗글에 대한 설명으로 적절하지 않은 것은?",
        "윗글의 내용과 일치하지 않는 것은?"
    ],
    "윗글의 내용 전개 방식으로 적절한 것은?": [
        "윗글의 내용 전개 방식으로 적절한 것은?"
    ],
    "윗글의 내용 전개 방식으로 가장 적절한 것은?": [
        "윗글의 내용 전개 방식으로 가장 적절한 것은?"
    ],
    "윗글의 내용 전개 방식으로 적절하지 않은 것은?": [
        "윗글의 내용 전개 방식으로 적절하지 않은 것은?"
    ],
    "윗글을 읽고 보인 반응으로 적절한 것은?": [
        "윗글을 읽고 보인 반응으로 적절한 것은?"
    ],
    "윗글을 읽고 보인 반응으로 가장 적절한 것은?": [
        "윗글을 읽고 보인 반응으로 가장 적절한 것은?"
    ],
    "윗글을 읽고 보인 반응으로 적절하지 않은 것은?": [
        "윗글을 읽고 보인 반응으로 적절하지 않은 것은?"
    ],
    "글에서 알 수 있는 'OOO'의 생각으로 적절한 것은?": [
        "글에서 알 수 있는 'OOO'의 생각으로 적절한 것은?"
    ],
    "글에서 알 수 있는 'OOO'의 생각으로 가장 적절한 것은?": [
        "글에서 알 수 있는 'OOO'의 생각으로 가장 적절한 것은?"
    ],
    "글에서 알 수 있는 'OOO'의 생각으로 적절하지 않은 것은?": [
        "글에서 알 수 있는 'OOO'의 생각으로 적절하지 않은 것은?"
    ],
    "밑줄에 해당하는 내용으로 적절한 것은?": [
        "밑줄에 해당하는 내용으로 적절한 것은?"
    ],
    "밑줄에 해당하는 내용으로 가장 적절한 것은?": [
        "밑줄에 해당하는 내용으로 가장 적절한 것은?"
    ],
    "밑줄에 해당하는 내용으로 적절하지 않은 것은?": [
        "밑줄에 해당하는 내용으로 적절하지 않은 것은?"
    ],
    "윗글을 바탕으로 <보기>를 이해한 내용으로 적절한 것은?": [
        "윗글을 바탕으로 <보기>를 이해한 내용으로 적절한 것은?"
    ],
    "윗글을 바탕으로 <보기>를 이해한 내용으로 가장 적절한 것은?": [
        "윗글을 바탕으로 <보기>를 이해한 내용으로 가장 적절한 것은?"
    ],
    "윗글을 바탕으로 <보기>를 이해한 내용으로 적절하지 않은 것은?": [
        "윗글을 바탕으로 <보기>를 이해한 내용으로 적절하지 않은 것은?"
    ],
    "필자의 관점에서 <보기>를 평가한 내용으로 적절한 것은?": [
        "필자의 관점에서 <보기>를 평가한 내용으로 적절한 것은?"
    ],
    "필자의 관점에서 <보기>를 평가한 내용으로 가장 적절한 것은?": [
        "필자의 관점에서 <보기>를 평가한 내용으로 가장 적절한 것은?"
    ],
    "필자의 관점에서 <보기>를 평가한 내용으로 적절하지 않은 것은?": [
        "필자의 관점에서 <보기>를 평가한 내용으로 적절하지 않은 것은?"
    ],
    "윗글을 읽고 추론한 내용으로 적절한 것은?": [
        "윗글을 읽고 추론한 내용으로 적절한 것은?",
        "윗글을 통해 알 수 있는 내용으로 적절한 것은?",
        "윗글에서 답을 찾을 수 있는 질문에 해당하는 것은?",
        "윗글에서 추론할 수 있는 것은?"
    ],
    "윗글을 읽고 추론한 내용으로 가장 적절한 것은?": [
        "윗글을 읽고 추론한 내용으로 가장 적절한 것은?",
        "윗글을 통해 알 수 있는 내용으로 가장 적절한 것은?"
    ],
    "윗글을 읽고 추론한 내용으로 적절하지 않은 것은?": [
        "윗글을 읽고 추론한 내용으로 적절하지 않은 것은?",
        "윗글을 통해 알 수 있는 내용으로 적절하지 않은 것은?"
    ],
    "(ㄱ)의 이유로 적절한 것은?": [
        "(ㄱ)의 이유로 적절한 것은?"
    ],
    "(ㄱ)의 이유로 가장 적절한 것은?": [
        "(ㄱ)의 이유로 가장 적절한 것은?"
    ],
    "(ㄱ)의 이유로 적절하지 않은 것은?": [
        "(ㄱ)의 이유로 적절하지 않은 것은?"
    ],
    "윗글을 읽고 (ㄱ)에 대해 보인 반응으로 적절한 것은?": [
        "윗글을 읽고 (ㄱ)에 대해 보인 반응으로 적절한 것은?"
    ],
    "윗글을 읽고 (ㄱ)에 대해 보인 반응으로 가장 적절한 것은?": [
        "윗글을 읽고 (ㄱ)에 대해 보인 반응으로 가장 적절한 것은?"
    ],
    "윗글을 읽고 (ㄱ)에 대해 보인 반응으로 적절하지 않은 것은?": [
        "윗글을 읽고 (ㄱ)에 대해 보인 반응으로 적절하지 않은 것은?"
    ],
    "문맥상 (ㄱ)과 바꾸어 쓰기에 가장 적절한 것은?": [
        "문맥상 (ㄱ)과 바꾸어 쓰기에 가장 적절한 것은?",
        "문맥상 (ㄱ)의 의미와 가장 가까운 것은?"
    ],
    "문맥상 (ㄱ)과 바꾸어 쓰기에 적절하지 않은 것은?": [
        "문맥상 (ㄱ)과 바꾸어 쓰기에 적절하지 않은 것은?"
    ]
}


In [18]:
def classify_question_type_fuzzy(question_text):
    """주어진 질문을 기준으로 가장 유사한 유형을 찾는 함수"""
    cleaned_text = question_text.strip()  # 앞뒤 공백 및 개행 문자 제거
    best_match = None
    highest_score = 0

    for category, patterns in question_type_criteria.items():
        score = process.extractOne(cleaned_text, patterns)  # 가장 유사한 패턴 찾기
        if score and score[1] > highest_score:  # 유사도가 가장 높은 경우
            highest_score = score[1]
            best_match = category
    
    return best_match if highest_score > 70 else "기타"  # 유사도가 70% 이상일 때만 반환

# JSON 데이터 업데이트 (올바른 함수 적용)
for entry in data:
    question_text = entry.get("question_text", "").strip()
    entry["question_type"] = classify_question_type_fuzzy(question_text)

In [19]:
# 결과 저장
output_json_path = "./information_added1.json"

with open(output_json_path, "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print(f"질문 유형 분류가 완료된 JSON이 저장되었습니다: {output_json_path}")

질문 유형 분류가 완료된 JSON이 저장되었습니다: ./information_added1.json


In [21]:
import json
from rapidfuzz import process, fuzz

# 파일 로드
file_path = "./information_added1.json"

with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)


def clean_text(text):
    """문자열을 소문자로 변환하고, 개행 및 불필요한 공백 제거"""
    text = text.replace("\n", " ").replace("\t", " ")  # 개행 문자 제거
    text = text.lower().strip()  # 소문자 변환 및 공백 제거
    return text

def get_first_two_sentences(text):
    """문장에서 처음 두 문장만 추출"""
    text = clean_text(text)
    sentences = text.split(". ")  # 문장을 '.' 기준으로 나눔
    return ". ".join(sentences[:2]) if len(sentences) > 1 else text  # 2문장까지만 유지

def classify_question_type_fuzzy(question_text):
    """기타로 분류된 질문을 다시 유사도 기반으로 매칭"""
    processed_text = get_first_two_sentences(question_text)  # 처음 두 문장만 사용
    best_match = None
    highest_score = 0

    for category, patterns in question_type_criteria.items():
        match, score, _ = process.extractOne(processed_text, [clean_text(p) for p in patterns], scorer=fuzz.WRatio)
        if score > highest_score:
            highest_score = score
            best_match = category

    return best_match if highest_score > 60 else "기타"  # 유사도 60% 이상이면 적용

# "기타"로 분류된 데이터만 다시 분류
for entry in data:
    if entry["question_type"] == "기타":
        question_text = entry.get("question_text", "").strip()
        new_question_type = classify_question_type_fuzzy(question_text)
        entry["question_type"] = new_question_type  # 새롭게 분류된 값 적용


# 업데이트된 JSON 파일 저장
updated_file_path = "./information_reclassified.json"

with open(updated_file_path, "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print(f'업데이트된 JSON 파일이 저장되었습니다: {updated_file_path}')


업데이트된 JSON 파일이 저장되었습니다: ./information_reclassified.json
